# Materi Klasifikasi Menggunakan Algoritma Naive Bayes

Naïve Bayes Classifier merupakan sebuah metoda klasifikasi yang berakar pada teorema Bayes . Metode pengklasifikasian dg menggunakan metode probabilitas dan statistik yg dikemukakan oleh ilmuwan Inggris Thomas Bayes , yaitu memprediksi peluang di masa depan berdasarkan pengalaman di masa sebelumnya sehingga dikenal sebagai Teorema Bayes . Ciri utama dr Naïve Bayes Classifier ini adalah asumsi yg sangat kuat (naïf) akan independensi dari masing-masing kondisi / kejadian.

Menurut Olson Delen (2008) menjelaskan Naïve Bayes unt setiap kelas keputusan, menghitung probabilitas dg syarat bahwa kelas keputusan adalah benar, mengingat vektor informasi obyek. Algoritma ini mengasumsikan bahwa atribut obyek adalah independen. Probabilitas yang terlibat dalam memproduksi perkiraan akhir dihitung sebagai jumlah frekuensi dr ” master ” tabel keputusan.

Naive Bayes Classifier bekerja sangat baik dibanding dengan model classifier lainnya. Hal ini dibuktikan oleh Xhemali , Hinde Stone dalam jurnalnya “Naïve Bayes vs. Decision Trees vs. Neural Networks in the Classification of Training Web Pages” mengatakan bahwa “Naïve Bayes Classifier memiliki tingkat akurasi yg lebih baik dibandingmodel classifier lainnya”.

Keuntungan penggunan adalah bahwa metoda ini hanya membutuhkan jumlah data pelatihan ( training data ) yg kecil unt menentukan estimasi parameter yg diperlukan dalam proses pengklasifikasian. Karena yg diasumsikan sebagai variable independent, maka hanya varians dr suatu variable dalam sebuah kelas yg dibutuhkan unt menentukan klasifikasi, bukan keseluruhan dr matriks kovarians.

## Kegunaan Naïve Bayes

- Mengklasifikasikan dokumen teks seperti teks berita ataupun teks akademis
- Sebagai metode machine learning yang menggunakan probabilitas
- Untuk membuat diagnosis medis secara otomatis
- Mendeteksi atau menyaring spam

## Kelebihan Naïve Bayes

- Bisa dipakai untuk data kuantitatif maupun kualitatif
- Tidak memerlukan jumlah data yang banyak
- Tidak perlu melakukan data training yang banyak
- Jika ada nilai yang hilang, maka bisa diabaikan dalam perhitungan.
- Perhitungannya cepat dan efisien
- Mudah dipahami
- Mudah dibuat
- Pengklasifikasian dokumen bisa dipersonalisasi, disesuaikan dengan kebutuhan setiap orang
- Jika digunakan dalaam bahasa pemrograman, code-nya sederhana
- Bisa digunakan untuk klasifikasi masalah biner ataupun multiclass

## Kekurangan Naïve Bayes

- Apabila probabilitas kondisionalnya bernilai nol, maka probabilitas prediksi juga akan bernilai nol
- Asumsi bahwa masing-masing variabel independen membuat berkurangnya akurasi, karena biasanya ada korelasi antara variabel yang satu dengan variabel yang lain
- Keakuratannya tidak bisa diukur menggunakan satu probabilitas saja. Butuh bukti-bukti lain untuk membuktikannya.
- Untuk membuat keputusan, diperlukan pengetahuan awal atau pengetahuan mengenai masa sebelumnya. Keberhasilannya sangat bergantung pada pengetahuan awal tersebut Banyak celah yang bisa mengurangi efektivitasnya Dirancang untuk mendeteksi kata-kata saja, tidak bisa berupa gambar

In [1]:
%pip install pymysql
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymysql
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore  # Import untuk deteksi outlier

def get_data_mysql():
    conn = pymysql.connect(
        host="mysql-761e1af-mysqlmysql.d.aivencloud.com",
        user="avnadmin",
        password="AVNS_b3-R_nwIM0G13-jogyl",
        database="defaultdb",
        port=22667
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM flowers")  # Ambil semua data dari tabel 'flowers'
    data = cursor.fetchall()

    conn.close()

    # Ubah data ke dalam bentuk NumPy array
    data_list = [list(row) for row in data]
    data_numpy = np.array(data_list)

    return data_numpy

# Ambil data dari MySQL
data = get_data_mysql()
print(data)

# Asumsikan kolom terakhir adalah label dan sisanya adalah fitur
X = data[:, :-1]  # Semua kolom kecuali yang terakhir sebagai fitur
y = data[:, -1]   # Kolom terakhir sebagai label

# --- Convert string features to numerical using Label Encoding ---
label_encoder = LabelEncoder()

X = X.astype(str)
for i in range(X.shape[1]):  # Iterate through all columns of X
    X[:, i] = label_encoder.fit_transform(X[:, i])

X = X.astype(int)

# Convert y to numerical if it's not already
if isinstance(y[0], str):
    y = label_encoder.fit_transform(y)

# --- End of conversion ---

# 1. Model Naïve Bayes dengan Data Asli
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_original = GaussianNB()
model_original.fit(X_train, y_train)
y_pred_original = model_original.predict(X_test)
accuracy_original = accuracy_score(y_test, y_pred_original)

print(f"\nAkurasi Model dengan Outlier: {accuracy_original:.2f}")

# 2. Mendeteksi dan Menghapus Outlier dengan Z-Score
z_scores = np.abs(zscore(X))  # Hitung Z-score
threshold = 3  # Ambang batas untuk mendeteksi outlier
mask = (z_scores < threshold).all(axis=1)  # True jika data tidak outlier

X_clean = X[mask]  # Simpan data tanpa outlier
y_clean = y[mask]

# 3. Model Naïve Bayes dengan Data Tanpa Outlier
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

model_clean = GaussianNB()
model_clean.fit(X_train_clean, y_train_clean)
y_pred_clean = model_clean.predict(X_test_clean)
accuracy_clean = accuracy_score(y_test_clean, y_pred_clean)

print(f"\nAkurasi Model Tanpa Outlier: {accuracy_clean:.2f}")

# 4. Perbandingan Akurasi
print("\n--- Perbandingan Akurasi ---")
print(f"Dengan Outlier  : {accuracy_original:.2f}")
print(f"Tanpa Outlier   : {accuracy_clean:.2f}")


[['1' 'Iris-setosa' '86.4' '70.0']
 ['2' 'Iris-setosa' '1.4' '0.2']
 ['3' 'Iris-setosa' '1.3' '0.2']
 ['4' 'Iris-setosa' '1.5' '0.2']
 ['5' 'Iris-setosa' '1.4' '0.2']
 ['6' 'Iris-setosa' '1.7' '0.4']
 ['7' 'Iris-setosa' '1.4' '0.3']
 ['8' 'Iris-setosa' '1.5' '0.2']
 ['9' 'Iris-setosa' '1.4' '0.2']
 ['10' 'Iris-setosa' '1.5' '0.1']
 ['11' 'Iris-setosa' '1.5' '0.2']
 ['12' 'Iris-setosa' '1.6' '0.2']
 ['13' 'Iris-setosa' '1.4' '0.1']
 ['14' 'Iris-setosa' '1.1' '0.1']
 ['15' 'Iris-setosa' '1.2' '0.2']
 ['16' 'Iris-setosa' '1.5' '0.4']
 ['17' 'Iris-setosa' '1.3' '0.4']
 ['18' 'Iris-setosa' '1.4' '0.3']
 ['19' 'Iris-setosa' '1.7' '0.3']
 ['20' 'Iris-setosa' '1.5' '0.3']
 ['21' 'Iris-setosa' '1.7' '0.2']
 ['22' 'Iris-setosa' '1.5' '0.4']
 ['23' 'Iris-setosa' '1.0' '0.2']
 ['24' 'Iris-setosa' '1.7' '0.5']
 ['25' 'Iris-setosa' '1.9' '0.2']
 ['26' 'Iris-setosa' '1.6' '0.2']
 ['27' 'Iris-setosa' '1.6' '0.4']
 ['28' 'Iris-setosa' '1.5' '0.2']
 ['29' 'Iris-setosa' '1.4' '0.2']
 ['30' 'Iris-setosa' 

In [3]:
import psycopg2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore  # Import untuk deteksi outlier

def get_pg_data():
    conn = psycopg2.connect(
        host="pg-23566fa8-poostgree.d.aivencloud.com",
        user="avnadmin",
        password="AVNS_PQHHNe5BmPwoXLKHHad",
        database="defaultdb",
        port=19790
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM rossi.postgree")  # Select all records from the table
    data = cursor.fetchall()  # Fetch all rows
    cursor.close()
    conn.close()

    # Convert to Python list
    data_list = [list(row) for row in data]

    # Convert to NumPy array
    data_numpy = np.array(data_list)

    return data_numpy # Return the numpy array instead of the list

# Ambil data dari MySQL
data = get_pg_data()
print(data)

# Asumsikan kolom terakhir adalah label dan sisanya adalah fitur
X = data[:, :-1]  # Semua kolom kecuali yang terakhir sebagai fitur
y = data[:, -1]   # Kolom terakhir sebagai label

# --- Convert string features to numerical using Label Encoding ---
label_encoder = LabelEncoder()

X = X.astype(str)
for i in range(X.shape[1]):  # Iterate through all columns of X
    X[:, i] = label_encoder.fit_transform(X[:, i])

X = X.astype(int)

# Convert y to numerical if it's not already
if isinstance(y[0], str):
    y = label_encoder.fit_transform(y)

# --- End of conversion ---

# 1. Model Naïve Bayes dengan Data Asli
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_original = GaussianNB()
model_original.fit(X_train, y_train)
y_pred_original = model_original.predict(X_test)
accuracy_original = accuracy_score(y_test, y_pred_original)

print(f"\nAkurasi Model dengan Outlier: {accuracy_original:.2f}")

# 2. Mendeteksi dan Menghapus Outlier dengan Z-Score
z_scores = np.abs(zscore(X))  # Hitung Z-score
threshold = 3  # Ambang batas untuk mendeteksi outlier
mask = (z_scores < threshold).all(axis=1)  # True jika data tidak outlier

X_clean = X[mask]  # Simpan data tanpa outlier
y_clean = y[mask]

# 3. Model Naïve Bayes dengan Data Tanpa Outlier
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

model_clean = GaussianNB()
model_clean.fit(X_train_clean, y_train_clean)
y_pred_clean = model_clean.predict(X_test_clean)
accuracy_clean = accuracy_score(y_test_clean, y_pred_clean)

print(f"\nAkurasi Model Tanpa Outlier: {accuracy_clean:.2f}")

# 4. Perbandingan Akurasi
print("\n--- Perbandingan Akurasi ---")
print(f"Dengan Outlier  : {accuracy_original:.2f}")
print(f"Tanpa Outlier   : {accuracy_clean:.2f}")

[['2' 'Iris-setosa' '4.9' '3.0']
 ['3' 'Iris-setosa' '4.7' '3.2']
 ['4' 'Iris-setosa' '4.6' '3.1']
 ['5' 'Iris-setosa' '5.0' '3.6']
 ['6' 'Iris-setosa' '5.4' '3.9']
 ['7' 'Iris-setosa' '4.6' '3.4']
 ['8' 'Iris-setosa' '5.0' '3.4']
 ['9' 'Iris-setosa' '4.4' '2.9']
 ['10' 'Iris-setosa' '4.9' '3.1']
 ['11' 'Iris-setosa' '5.4' '3.7']
 ['12' 'Iris-setosa' '4.8' '3.4']
 ['13' 'Iris-setosa' '4.8' '3.0']
 ['14' 'Iris-setosa' '4.3' '3.0']
 ['15' 'Iris-setosa' '5.8' '4.0']
 ['16' 'Iris-setosa' '5.7' '4.4']
 ['17' 'Iris-setosa' '5.4' '3.9']
 ['18' 'Iris-setosa' '5.1' '3.5']
 ['19' 'Iris-setosa' '5.7' '3.8']
 ['20' 'Iris-setosa' '5.1' '3.8']
 ['21' 'Iris-setosa' '5.4' '3.4']
 ['22' 'Iris-setosa' '5.1' '3.7']
 ['23' 'Iris-setosa' '4.6' '3.6']
 ['24' 'Iris-setosa' '5.1' '3.3']
 ['25' 'Iris-setosa' '4.8' '3.4']
 ['26' 'Iris-setosa' '5.0' '3.0']
 ['27' 'Iris-setosa' '5.0' '3.4']
 ['28' 'Iris-setosa' '5.2' '3.5']
 ['29' 'Iris-setosa' '5.2' '3.4']
 ['30' 'Iris-setosa' '4.7' '3.2']
 ['31' 'Iris-setosa' '